In [61]:
import pandas as pd
import os
from os import path as osp
import numpy as np

In [161]:
def get_lemma_dict(path="data/lemma_dict.txt"):
    lemma_dict = dict()
    with open(path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            l = line.split()
            lemma_dict[l[0]] = l[1]
    return lemma_dict

def get_stop_words(path="data/stop_words.txt"):
    stop_words = set()
    with open(path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line[0] != '*':
                stop_words.add(line.strip())
    return stop_words

In [162]:
lemma_dict = get_lemma_dict()

In [163]:
stop_words = get_stop_words()

In [127]:
def get_ascii(word):
    l = "абвгґдеєжзиіїйклмнопрстуфхцчшщьюя-\'"
    s = "!?.;\"'/\\,;()"
    new = ""
    for w in word:
        if w in l:
            new += w
        elif w in s and (new and new[-1] != ' '):
            new += " "
    return new

def get_lemma_word(word, ld=lemma_dict, sw=stop_words):
    new_word = get_ascii(word.lower().strip())
    words = [x.strip() for x in new_word.split()]
    if len(words) <= 1:
        if new_word and new_word in ld and new_word not in stop_words:
            return [ld[new_word]]
    else:
        res = []
        for word in words:
            if word and word in ld and word not in stop_words:
                res.append(ld[word])
        return res
    return [""]

def get_lemma_sent(sent, ld=lemma_dict):
    new = []
   
    for word in sent.split():
        word = get_lemma_word(word)
        if word and word != [""]: 
            for w in word:
                new.append(w)
    return new

def get_lemma_par(par):
    new = []
    for sent in par.split('.'):
        sent = get_lemma_sent(sent)
        if sent: 
            new.append(sent)
    return new

In [171]:
get_lemma_par("ДУПА СаЛо в й а рило тобі sjlghs jklhgjkls hgkls hdgkl тупи..й дебіл. гниле!!!!дерево і???(стільця) не варте.\n\\\/n//навіть реп'яхів")

[['дупа', 'сало', 'рити', 'тупити'],
 ['дебіл'],
 ['гниль', 'дерево', 'стілець', 'не', 'вартий'],
 ['навіть', "реп'ях"]]

In [199]:
d = {"neutral":0, "negative":-1, "positive":1}
data_path = "/home/zabulskyy/data/coded_articles/ukr_text_Kolya/"

In [200]:
composed_data = pd.DataFrame({"tone":list(), "text":list()})
chunks = []
for filename in os.listdir(data_path):
    datai = pd.read_csv(osp.join(data_path, filename),  encoding = "utf-8",  lineterminator='\n').dropna()
    temp = pd.DataFrame({"tone":datai["keyword_tone"].apply(lambda x: d[x]), "text":datai["text"].apply(get_lemma_sent).apply(lambda x: ' '.join(x))})
    chunks.append(temp)
composed_data = pd.concat(chunks, ignore_index=1).drop_duplicates()

In [201]:
len(composed_data)

46962

In [202]:
composed_data.head()

,tone,text
0,0,препарат его первий не не не оно же состав фор...
1,-1,сайт служба контроль наркотик от заперти ораль...
3,-1,служба контроль наркотик об том сайт один орал...
4,-1,сайт служба контроль наркотик от заперти мл ам...
6,-1,розпорядження надходження інформація правоохор...


In [203]:
ukr_data = pd.read_csv("data/ukrainian_reviews_corpus.csv", sep="|")

In [212]:
ukr_data.head()

,opinion_rating,opinion_text
0,1,хороший місце провести кальян смачний кожний г...
1,1,взуття брати не мінус додзвонитися взагалі не
2,1,відмінний магазин чемний єдиний зауваження отр...
3,1,замовляти відправити
4,1,замовляти втретє магазин планувати замовляти п...


In [205]:
ukr_data["opinion_rating"] = ukr_data["opinion_rating"].apply(lambda x: 1 if x == 0 else -1)

In [206]:
ukr_data = ukr_data.drop_duplicates()

In [207]:
ukr_data["opinion_text"] = ukr_data["opinion_text"].apply(lambda x: str(x))

In [211]:
ukr_data["opinion_text"] = ukr_data["opinion_text"].apply(get_lemma_sent).apply(lambda x: ' '.join(x))

In [213]:
composed_data = pd.concat([composed_data, pd.DataFrame({"tone":ukr_data["opinion_rating"], "text":ukr_data["opinion_text"]})], ignore_index=1)

In [219]:
composed_data = composed_data.drop_duplicates()

In [222]:
composed_data.to_csv("data/train.csv", index=False)

In [230]:
len(composed_data[composed_data["tone"] == -1]), len(composed_data[composed_data["tone"] == 0]), len(composed_data[composed_data["tone"] == 1])

(22805, 17782, 29396)